In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import albumentations as A
import torch
import torch.nn as nn
from torchvision.models.segmentation import fcn_resnet50
from albumentations.pytorch import ToTensorV2
from pathlib import Path

> ## 사용법

1. `./data`로 current working directory 변경
2. train 데이터 바꾸고 싶으면 `train_dir = True`, eval 데이터 바꾸고 싶으면 `train_dir = False`
2. 코드 실행
3. `./data/train/images_segmented/...`에 segmented 이미지들 원래랑 똑같은 구조로 저장됨

In [2]:
train_dir = False
target_dir = 'train' if train_dir else 'eval'

In [3]:
segmented_img_dir_path = '/opt/ml/input/data/{}/images_seg'.format(target_dir)
orig_img_dir_path = '/opt/ml/input/data/{}/images'.format(target_dir)
if not os.path.isdir(segmented_img_dir_path):
    os.mkdir(segmented_img_dir_path)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [5]:
model = fcn_resnet50(pretrained=True, progress=False).to(device)
model = model.eval()

In [6]:
def segment_img(img):
    transform = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2(),
    ])
    
    original_img = img

    img = transform(image=np.array(img))['image']
    img_batch = img.unsqueeze(0).to(device).float()

    output = model(img_batch)['out']
    normalized_masks = torch.nn.functional.softmax(output, dim=1)

    person_mask = normalized_masks[0,15]
    masked_img = person_mask.detach().cpu().numpy()
    
    boundary = 5
    xmax = masked_img.shape[1]
    ymax = masked_img.shape[0]
    
    masked_img = masked_img > 0.9
    temp_masked_img = masked_img.copy()

    
    for i in range(ymax):
        for j in range(xmax):
            if temp_masked_img[i,j] == True:
                x_left, x_right = max(0,j-boundary), min(xmax-1, j+boundary)
                y_down, y_up = max(0,i-boundary), min(ymax-1, i+boundary)
                masked_img[y_down:y_up, x_left:x_right] = 1

#     original_img = A.Resize(224,224)(image=np.array(original_img))['image']
    masked_img = np.expand_dims(masked_img, axis = 2)
    masked_img = np.concatenate((masked_img, masked_img, masked_img), axis=2)
    
    final_img = original_img * masked_img
    
    for i in range(ymax):
        for j in range(xmax):
            if(final_img[i][j][0] == 0 and final_img[i][j][1] == 0 and final_img[i][j][2] == 0):
                final_img[i][j][0] = 255
                final_img[i][j][1] = 53
                final_img[i][j][2] = 184 
    
    return final_img

In [12]:

def train_segment():
    count = 0
    for person_dir in os.listdir(orig_img_dir_path):
        if person_dir.startswith('.'):
            continue
        for fname in os.listdir(Path(orig_img_dir_path) / person_dir):
            if fname.startswith('.'):
                continue
            img_full_path = Path(orig_img_dir_path) / person_dir / fname
            img = Image.open(img_full_path)
            segmented_img = Image.fromarray(segment_img(img))

            new_dir = Path(segmented_img_dir_path) / person_dir
            if not os.path.isdir(new_dir):
                os.makedirs(new_dir)
            new_path = new_dir / fname
            segmented_img.save(new_path)
        
        if (count % 100 == 0):
            print(count)
        count += 1

def eval_segment():
    count = 0
    for fname in os.listdir(Path(orig_img_dir_path)):
        if fname.startswith('.'):
            continue
        img_full_path = Path(orig_img_dir_path) / fname
        img = Image.open(img_full_path)
        segmented_img = Image.fromarray(segment_img(img))
        
        new_dir = Path(segmented_img_dir_path)
        if not os.path.isdir(new_dir):
            os.makedirs(new_dir)
        
        new_path = new_dir / fname
        segmented_img.save(new_path)
        if (count % 100 == 0):
            print(count)
        count += 1

In [ ]:
if train_dir:
    train_segment()
else:
    eval_segment()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
